In [1]:
import xarray as xr # for data handling
import numpy as np # for maths
import matplotlib as mpl # plotlibrary
import matplotlib.pyplot as plt # for the plots 
from matplotlib.patches import Rectangle
from matplotlib.colors import LogNorm # for making a log 10 colorscale
import cartopy # library for making maps
from cartopy import crs as ccrs # Projection Library
from cartopy.crs import Stereographic, PlateCarree, Mollweide # Map projections I use00
from cartopy.feature import LAND, COASTLINE # Vectordata of the Coastline and land for making the maps prettier
import cmocean
import gsw


[March File](https://www.pmel.noaa.gov/mimoc/data/MIMOC_Z_GRID_v2.2_CT_SA_month03.nc.gz)

[September File](https://www.pmel.noaa.gov/mimoc/data/MIMOC_Z_GRID_v2.2_CT_SA_month09.nc.gz)

In [14]:
march_file = "MIMOC_Z_GRID_v2.2_CT_SA_month03.nc"
march = xr.open_dataset(march_file, decode_times=False)
march = march.set_coords([
        "LATITUDE", "LONGITUDE", "PRESSURE",
    ]).set_index(indexes={
        "LAT":"LATITUDE", "LONG": "LONGITUDE", "PRES": "PRESSURE",
    }).rename_dims({
        "LAT":"lat", "LONG":"lon", "PRES":"pressure"
    }).rename_vars({
        "LAT":"lat", "LONG":"lon", "PRES":"pressure"
    })
march.info

<bound method Dataset.info of <xarray.Dataset> Size: 159MB
Dimensions:                   (pressure: 81, lat: 341, lon: 720)
Coordinates:
  * pressure                  (pressure) float32 324B 0.0 5.0 ... 1.95e+03
  * lat                       (lat) float32 1kB -80.0 -79.5 -79.0 ... 89.5 90.0
  * lon                       (lon) float32 3kB 0.0 0.5 1.0 ... 359.0 359.5
Data variables:
    ABSOLUTE_SALINITY         (pressure, lat, lon) float32 80MB ...
    CONSERVATIVE_TEMPERATURE  (pressure, lat, lon) float32 80MB ...>

In [15]:
sept_file = "MIMOC_Z_GRID_v2.2_CT_SA_month09.nc"
sept = xr.open_dataset(sept_file, decode_times=False)
sept = sept.set_coords([
        "LATITUDE", "LONGITUDE", "PRESSURE",
    ]).set_index(indexes={
        "LAT":"LATITUDE", "LONG": "LONGITUDE", "PRES": "PRESSURE",
    }).rename_dims({
        "LAT":"lat", "LONG":"lon", "PRES":"pressure"
    }).rename_vars({
        "LAT":"lat", "LONG":"lon", "PRES":"pressure"
    })
sept.info

<bound method Dataset.info of <xarray.Dataset> Size: 159MB
Dimensions:                   (pressure: 81, lat: 341, lon: 720)
Coordinates:
  * pressure                  (pressure) float32 324B 0.0 5.0 ... 1.95e+03
  * lat                       (lat) float32 1kB -80.0 -79.5 -79.0 ... 89.5 90.0
  * lon                       (lon) float32 3kB 0.0 0.5 1.0 ... 359.0 359.5
Data variables:
    ABSOLUTE_SALINITY         (pressure, lat, lon) float32 80MB ...
    CONSERVATIVE_TEMPERATURE  (pressure, lat, lon) float32 80MB ...>

North Atlantic: 10deg N to 50deg N by 70deg W to 0deg E

South Atlantic: 10deg S to 50deg S by 70deg W to 0deg E

In [ ]:
north_sal = march.ABSOLUTE_SALINITY.sel(lat=)

array(-15., dtype=float32)